In [ ]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configuración de visualización
plt.style.use('seaborn')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

# Configurar visualización de datos en el notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', lambda x: '{:,.0f}'.format(x) if isinstance(x, (int, float)) else str(x))

# Configurar estilo de gráficos
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

# Configurar formato de números en español
import locale
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')


In [ ]:
# Cargar datos procesados
try:
    print("Intentando cargar datos procesados...")
    datos = pd.read_pickle('datos_procesados.pkl')
except FileNotFoundError:
    print("Datos procesados no encontrados. Procesando datos desde archivos originales...")
    from preparar_datos_eph import cargar_datos
    datos = cargar_datos()

# Mostrar información básica del dataset
print("\nInformación del dataset:")
print(datos.info())

# Mostrar estadísticas descriptivas de variables numéricas
print("\nEstadísticas descriptivas de variables numéricas:")
print(datos[['edad', 'ingresos', 'ocupados_hogar']].describe())

# Mostrar distribuciones de variables categóricas
print("\nDistribución de variables categóricas:")
for col in ['nivel_educativo', 'estado_actividad', 'sexo', 'region', 
            'compra_cuotas', 'prestamo_personas', 'prestamo_banco']:
    print(f"\n{col.replace('_', ' ').title()}:")
    print(datos[col].value_counts(normalize=True).apply(lambda x: f"{x:.1%}"))


In [ ]:
# Probabilidades simples
def calcular_probabilidades_simples(datos):
    resultados = {}
    
    # P(Nivel educativo)
    resultados['p_nivel_educ'] = datos['nivel_educativo'].value_counts(normalize=True)
    
    # P(Estado de actividad)
    resultados['p_estado_act'] = datos['estado_actividad'].value_counts(normalize=True)
    
    # P(Acceso a crédito)
    resultados['p_credito'] = {
        'Compra en cuotas': (datos['compra_cuotas'] == 'Sí').mean(),
        'Préstamo personal': (datos['prestamo_personas'] == 'Sí').mean(),
        'Préstamo bancario': (datos['prestamo_banco'] == 'Sí').mean()
    }
    
    # P(Región)
    resultados['p_region'] = datos['region'].value_counts(normalize=True)
    
    # P(Universitario Y Desocupado)
    resultados['p_univ_desoc'] = ((datos['nivel_educativo'] == 'Superior universitario completo') & 
                                 (datos['estado_actividad'] == 'Desocupado')).mean()
    
    return resultados

# Calcular probabilidades
prob_simples = calcular_probabilidades_simples(datos)

# Crear visualizaciones
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Nivel educativo
prob_simples['p_nivel_educ'].plot(kind='bar', ax=ax1)
ax1.set_title('Distribución de Nivel Educativo')
ax1.set_xlabel('Nivel Educativo')
ax1.set_ylabel('Probabilidad')
ax1.tick_params(axis='x', rotation=45)

# 2. Estado de actividad
prob_simples['p_estado_act'].plot(kind='bar', ax=ax2)
ax2.set_title('Distribución de Estado de Actividad')
ax2.set_xlabel('Estado')
ax2.set_ylabel('Probabilidad')
ax2.tick_params(axis='x', rotation=45)

# 3. Acceso a crédito
pd.Series(prob_simples['p_credito']).plot(kind='bar', ax=ax3)
ax3.set_title('Probabilidad de Acceso a Crédito')
ax3.set_xlabel('Tipo de Crédito')
ax3.set_ylabel('Probabilidad')
ax3.tick_params(axis='x', rotation=45)

# 4. Región
prob_simples['p_region'].plot(kind='bar', ax=ax4)
ax4.set_title('Distribución por Región')
ax4.set_xlabel('Región')
ax4.set_ylabel('Probabilidad')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Mostrar resultados numéricos
print("Probabilidades por nivel educativo:")
print(prob_simples['p_nivel_educ'].apply(lambda x: f"{x:.1%}"))

print("\nProbabilidades por estado de actividad:")
print(prob_simples['p_estado_act'].apply(lambda x: f"{x:.1%}"))

print("\nProbabilidades de acceso a crédito:")
for tipo, prob in prob_simples['p_credito'].items():
    print(f"{tipo}: {prob:.1%}")

print("\nProbabilidades por región:")
print(prob_simples['p_region'].apply(lambda x: f"{x:.1%}"))

print(f"\nP(Universitario ∩ Desocupado): {prob_simples['p_univ_desoc']:.1%}")


In [ ]:
# Probabilidades condicionales
def calcular_probabilidades_condicionales(datos):
    resultados = {}
    
    # P(Estado actividad | Nivel educativo)
    resultados['p_act_educ'] = pd.crosstab(
        datos['nivel_educativo'], 
        datos['estado_actividad'], 
        normalize='index'
    )
    
    # P(Acceso crédito | Nivel educativo)
    resultados['p_cred_educ'] = pd.DataFrame({
        'Compra en cuotas': datos.groupby('nivel_educativo')['compra_cuotas'].apply(lambda x: (x == 'Sí').mean()),
        'Préstamo personal': datos.groupby('nivel_educativo')['prestamo_personas'].apply(lambda x: (x == 'Sí').mean()),
        'Préstamo bancario': datos.groupby('nivel_educativo')['prestamo_banco'].apply(lambda x: (x == 'Sí').mean())
    })
    
    # P(Estado actividad | Región)
    resultados['p_act_region'] = pd.crosstab(
        datos['region'], 
        datos['estado_actividad'], 
        normalize='index'
    )
    
    # P(Acceso crédito | Estado actividad)
    resultados['p_cred_act'] = pd.DataFrame({
        'Compra en cuotas': datos.groupby('estado_actividad')['compra_cuotas'].apply(lambda x: (x == 'Sí').mean()),
        'Préstamo personal': datos.groupby('estado_actividad')['prestamo_personas'].apply(lambda x: (x == 'Sí').mean()),
        'Préstamo bancario': datos.groupby('estado_actividad')['prestamo_banco'].apply(lambda x: (x == 'Sí').mean())
    })
    
    return resultados

# Calcular probabilidades condicionales
prob_cond = calcular_probabilidades_condicionales(datos)

# Visualizaciones
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Estado de actividad por nivel educativo
prob_cond['p_act_educ'].plot(kind='bar', stacked=True, ax=ax1)
ax1.set_title('Estado de Actividad por Nivel Educativo')
ax1.set_xlabel('Nivel Educativo')
ax1.set_ylabel('Probabilidad')
ax1.tick_params(axis='x', rotation=45)
ax1.legend(title='Estado de Actividad', bbox_to_anchor=(1.05, 1))

# 2. Acceso a crédito por nivel educativo
prob_cond['p_cred_educ'].plot(kind='bar', ax=ax2)
ax2.set_title('Acceso a Crédito por Nivel Educativo')
ax2.set_xlabel('Nivel Educativo')
ax2.set_ylabel('Probabilidad')
ax2.tick_params(axis='x', rotation=45)
ax2.legend(title='Tipo de Crédito', bbox_to_anchor=(1.05, 1))

# 3. Estado de actividad por región
prob_cond['p_act_region'].plot(kind='bar', stacked=True, ax=ax3)
ax3.set_title('Estado de Actividad por Región')
ax3.set_xlabel('Región')
ax3.set_ylabel('Probabilidad')
ax3.tick_params(axis='x', rotation=45)
ax3.legend(title='Estado de Actividad', bbox_to_anchor=(1.05, 1))

# 4. Acceso a crédito por estado de actividad
prob_cond['p_cred_act'].plot(kind='bar', ax=ax4)
ax4.set_title('Acceso a Crédito por Estado de Actividad')
ax4.set_xlabel('Estado de Actividad')
ax4.set_ylabel('Probabilidad')
ax4.tick_params(axis='x', rotation=45)
ax4.legend(title='Tipo de Crédito', bbox_to_anchor=(1.05, 1))

plt.tight_layout()
plt.show()

# Mostrar algunos resultados numéricos interesantes
print("P(Estado de actividad | Nivel educativo superior completo):")
print(prob_cond['p_act_educ'].loc['Superior universitario completo'].apply(lambda x: f"{x:.1%}"))

print("\nP(Acceso a crédito | Ocupado):")
for tipo in prob_cond['p_cred_act'].columns:
    print(f"{tipo}: {prob_cond['p_cred_act'].loc['Ocupado', tipo]:.1%}")

print("\nP(Estado de actividad | GBA):")
print(prob_cond['p_act_region'].loc['GBA'].apply(lambda x: f"{x:.1%}"))


In [ ]:
# Pruebas de independencia
def pruebas_independencia(datos):
    resultados = {}
    
    # Chi-cuadrado: Nivel educativo vs Estado de actividad
    tabla_educ_act = pd.crosstab(datos['nivel_educativo'], datos['estado_actividad'])
    chi2_educ_act, p_educ_act = stats.chi2_contingency(tabla_educ_act)[:2]
    resultados['chi2_educ_act'] = {'estadistico': chi2_educ_act, 'p_valor': p_educ_act}
    
    # Chi-cuadrado: Región vs Estado de actividad
    tabla_region_act = pd.crosstab(datos['region'], datos['estado_actividad'])
    chi2_region_act, p_region_act = stats.chi2_contingency(tabla_region_act)[:2]
    resultados['chi2_region_act'] = {'estadistico': chi2_region_act, 'p_valor': p_region_act}
    
    # Chi-cuadrado: Nivel educativo vs Acceso a crédito
    for credito in ['compra_cuotas', 'prestamo_personas', 'prestamo_banco']:
        tabla_educ_cred = pd.crosstab(datos['nivel_educativo'], datos[credito])
        chi2_educ_cred, p_educ_cred = stats.chi2_contingency(tabla_educ_cred)[:2]
        resultados[f'chi2_educ_{credito}'] = {'estadistico': chi2_educ_cred, 'p_valor': p_educ_cred}
    
    # Correlación entre variables numéricas
    vars_num = ['edad', 'ingresos', 'ocupados_hogar']
    resultados['correlacion'] = datos[vars_num].corr()
    
    return resultados

# Realizar pruebas
pruebas = pruebas_independencia(datos)

# Visualizaciones
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Mapa de calor de correlaciones
sns.heatmap(pruebas['correlacion'], annot=True, cmap='coolwarm', center=0, ax=ax1)
ax1.set_title('Correlación entre Variables Numéricas')

# 2. Nivel educativo vs Estado de actividad
sns.heatmap(pd.crosstab(datos['nivel_educativo'], datos['estado_actividad'], normalize='index'),
            annot=True, fmt='.2%', cmap='YlOrRd', ax=ax2)
ax2.set_title('Nivel Educativo vs Estado de Actividad')
ax2.tick_params(axis='x', rotation=45)
ax2.tick_params(axis='y', rotation=45)

# 3. Región vs Estado de actividad
sns.heatmap(pd.crosstab(datos['region'], datos['estado_actividad'], normalize='index'),
            annot=True, fmt='.2%', cmap='YlOrRd', ax=ax3)
ax3.set_title('Región vs Estado de Actividad')
ax3.tick_params(axis='x', rotation=45)

# 4. Nivel educativo vs Acceso a crédito
credito_educ = pd.DataFrame({
    'Compra en cuotas': datos.groupby('nivel_educativo')['compra_cuotas'].apply(lambda x: (x == 'Sí').mean()),
    'Préstamo personal': datos.groupby('nivel_educativo')['prestamo_personas'].apply(lambda x: (x == 'Sí').mean()),
    'Préstamo bancario': datos.groupby('nivel_educativo')['prestamo_banco'].apply(lambda x: (x == 'Sí').mean())
})
sns.heatmap(credito_educ, annot=True, fmt='.2%', cmap='YlOrRd', ax=ax4)
ax4.set_title('Nivel Educativo vs Acceso a Crédito')
ax4.tick_params(axis='x', rotation=45)
ax4.tick_params(axis='y', rotation=45)

plt.tight_layout()
plt.show()

# Mostrar resultados de pruebas de independencia
print("Resultados de pruebas Chi-cuadrado:")
print("\nNivel educativo vs Estado de actividad:")
print(f"Chi2 = {pruebas['chi2_educ_act']['estadistico']:.2f}, p-valor = {pruebas['chi2_educ_act']['p_valor']:.2e}")

print("\nRegión vs Estado de actividad:")
print(f"Chi2 = {pruebas['chi2_region_act']['estadistico']:.2f}, p-valor = {pruebas['chi2_region_act']['p_valor']:.2e}")

print("\nNivel educativo vs Acceso a crédito:")
for credito in ['compra_cuotas', 'prestamo_personas', 'prestamo_banco']:
    print(f"{credito}:")
    print(f"Chi2 = {pruebas[f'chi2_educ_{credito}']['estadistico']:.2f}, p-valor = {pruebas[f'chi2_educ_{credito}']['p_valor']:.2e}")


In [ ]:
# Aplicación del Teorema de Bayes
def calcular_probabilidades_bayes(datos):
    resultados = {}
    
    # P(Nivel educativo | Estado de actividad)
    tabla_educ_act = pd.crosstab(datos['nivel_educativo'], datos['estado_actividad'])
    total = len(datos)
    
    # Calcular probabilidades marginales
    p_educ = tabla_educ_act.sum(axis=1) / total
    p_act = tabla_educ_act.sum(axis=0) / total
    
    # Calcular P(Nivel educativo | Estado de actividad)
    p_educ_dado_act = pd.DataFrame(0.0, 
                                  index=tabla_educ_act.index,
                                  columns=tabla_educ_act.columns)
    
    for educ in tabla_educ_act.index:
        for act in tabla_educ_act.columns:
            # P(B|A): Prob de actividad dado nivel educativo
            p_act_dado_educ = tabla_educ_act.loc[educ, act] / tabla_educ_act.loc[educ].sum()
            # Teorema de Bayes
            p_educ_dado_act.loc[educ, act] = (p_act_dado_educ * p_educ[educ]) / p_act[act]
    
    resultados['p_educ_dado_act'] = p_educ_dado_act.astype(float)
    
    # P(Estado de actividad | Acceso a crédito)
    for credito in ['compra_cuotas', 'prestamo_personas', 'prestamo_banco']:
        tabla_act_cred = pd.crosstab(datos['estado_actividad'], datos[credito])
        
        # Calcular probabilidades marginales
        p_act = tabla_act_cred.sum(axis=1) / total
        p_cred = tabla_act_cred.sum(axis=0) / total
        
        p_act_dado_cred = pd.DataFrame(0.0,
                                     index=tabla_act_cred.index,
                                     columns=tabla_act_cred.columns)
        
        for act in tabla_act_cred.index:
            for cred in tabla_act_cred.columns:
                p_cred_dado_act = tabla_act_cred.loc[act, cred] / tabla_act_cred.loc[act].sum()
                p_act_dado_cred.loc[act, cred] = (p_cred_dado_act * p_act[act]) / p_cred[cred]
        
        resultados[f'p_act_dado_{credito}'] = p_act_dado_cred.astype(float)
    
    # P(Región | Estado de actividad)
    tabla_region_act = pd.crosstab(datos['region'], datos['estado_actividad'])
    
    # Calcular probabilidades marginales
    p_region = tabla_region_act.sum(axis=1) / total
    p_act = tabla_region_act.sum(axis=0) / total
    
    p_region_dado_act = pd.DataFrame(0.0,
                                   index=tabla_region_act.index,
                                   columns=tabla_region_act.columns)
    
    for region in tabla_region_act.index:
        for act in tabla_region_act.columns:
            p_act_dado_region = tabla_region_act.loc[region, act] / tabla_region_act.loc[region].sum()
            p_region_dado_act.loc[region, act] = (p_act_dado_region * p_region[region]) / p_act[act]
    
    resultados['p_region_dado_act'] = p_region_dado_act.astype(float)
    
    return resultados

# Calcular probabilidades de Bayes
prob_bayes = calcular_probabilidades_bayes(datos)

# Configurar el estilo de las visualizaciones
plt.style.use('seaborn')
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. P(Nivel educativo | Estado de actividad)
sns.heatmap(prob_bayes['p_educ_dado_act'], 
            annot=True, 
            fmt='.1%', 
            cmap='YlOrRd', 
            ax=ax1,
            cbar_kws={'format': '%.0f%%'})
ax1.set_title('P(Nivel Educativo | Estado de Actividad)')
ax1.tick_params(axis='x', rotation=45)
ax1.tick_params(axis='y', rotation=45)

# 2. P(Estado de actividad | Compra en cuotas)
sns.heatmap(prob_bayes['p_act_dado_compra_cuotas'], 
            annot=True, 
            fmt='.1%', 
            cmap='YlOrRd', 
            ax=ax2,
            cbar_kws={'format': '%.0f%%'})
ax2.set_title('P(Estado de Actividad | Compra en Cuotas)')
ax2.tick_params(axis='x', rotation=45)
ax2.tick_params(axis='y', rotation=45)

# 3. P(Estado de actividad | Préstamo bancario)
sns.heatmap(prob_bayes['p_act_dado_prestamo_banco'], 
            annot=True, 
            fmt='.1%', 
            cmap='YlOrRd', 
            ax=ax3,
            cbar_kws={'format': '%.0f%%'})
ax3.set_title('P(Estado de Actividad | Préstamo Bancario)')
ax3.tick_params(axis='x', rotation=45)
ax3.tick_params(axis='y', rotation=45)

# 4. P(Región | Estado de actividad)
sns.heatmap(prob_bayes['p_region_dado_act'], 
            annot=True, 
            fmt='.1%', 
            cmap='YlOrRd', 
            ax=ax4,
            cbar_kws={'format': '%.0f%%'})
ax4.set_title('P(Región | Estado de Actividad)')
ax4.tick_params(axis='x', rotation=45)
ax4.tick_params(axis='y', rotation=45)

plt.tight_layout()
plt.show()

# Mostrar algunos resultados interesantes
print("\nProbabilidades condicionales destacadas:")
print("\nP(Nivel educativo superior completo | Ocupado):")
print(f"{prob_bayes['p_educ_dado_act'].loc['Superior universitario completo', 'Ocupado']:.1%}")

print("\nP(Ocupado | Tiene préstamo bancario):")
print(f"{prob_bayes['p_act_dado_prestamo_banco'].loc['Ocupado', 'Sí']:.1%}")

print("\nP(Pampeana | Desocupado):")
print(f"{prob_bayes['p_region_dado_act'].loc['Pampeana', 'Desocupado']:.1%}")


In [ ]:
# Aplicación del Teorema de Bayes
def calcular_probabilidades_bayes(datos):
    resultados = {}
    
    # P(Nivel educativo | Estado de actividad)
    p_educ = datos['nivel_educativo'].value_counts(normalize=True)
    p_act = datos['estado_actividad'].value_counts(normalize=True)
    p_act_dado_educ = pd.crosstab(datos['nivel_educativo'], datos['estado_actividad'], normalize='index')
    
    p_educ_dado_act = pd.DataFrame(index=p_educ.index, columns=p_act.index)
    for educ in p_educ.index:
        for act in p_act.index:
            p_educ_dado_act.loc[educ, act] = (p_act_dado_educ.loc[educ, act] * p_educ[educ]) / p_act[act]
    
    resultados['p_educ_dado_act'] = p_educ_dado_act
    
    # P(Estado de actividad | Acceso a crédito)
    for credito in ['compra_cuotas', 'prestamo_personas', 'prestamo_banco']:
        p_act = datos['estado_actividad'].value_counts(normalize=True)
        p_cred = datos[credito].value_counts(normalize=True)
        p_cred_dado_act = pd.crosstab(datos['estado_actividad'], datos[credito], normalize='index')
        
        p_act_dado_cred = pd.DataFrame(index=p_act.index, columns=p_cred.index)
        for act in p_act.index:
            for cred in p_cred.index:
                p_act_dado_cred.loc[act, cred] = (p_cred_dado_act.loc[act, cred] * p_act[act]) / p_cred[cred]
        
        resultados[f'p_act_dado_{credito}'] = p_act_dado_cred
    
    # P(Región | Estado de actividad)
    p_region = datos['region'].value_counts(normalize=True)
    p_act = datos['estado_actividad'].value_counts(normalize=True)
    p_act_dado_region = pd.crosstab(datos['region'], datos['estado_actividad'], normalize='index')
    
    p_region_dado_act = pd.DataFrame(index=p_region.index, columns=p_act.index)
    for region in p_region.index:
        for act in p_act.index:
            p_region_dado_act.loc[region, act] = (p_act_dado_region.loc[region, act] * p_region[region]) / p_act[act]
    
    resultados['p_region_dado_act'] = p_region_dado_act
    
    return resultados

# Calcular probabilidades de Bayes
prob_bayes = calcular_probabilidades_bayes(datos)

# Visualizaciones
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. P(Nivel educativo | Estado de actividad)
sns.heatmap(prob_bayes['p_educ_dado_act'], annot=True, fmt='.2%', cmap='YlOrRd', ax=ax1)
ax1.set_title('P(Nivel Educativo | Estado de Actividad)')
ax1.tick_params(axis='x', rotation=45)
ax1.tick_params(axis='y', rotation=45)

# 2. P(Estado de actividad | Compra en cuotas)
sns.heatmap(prob_bayes['p_act_dado_compra_cuotas'], annot=True, fmt='.2%', cmap='YlOrRd', ax=ax2)
ax2.set_title('P(Estado de Actividad | Compra en Cuotas)')
ax2.tick_params(axis='x', rotation=45)
ax2.tick_params(axis='y', rotation=45)

# 3. P(Estado de actividad | Préstamo bancario)
sns.heatmap(prob_bayes['p_act_dado_prestamo_banco'], annot=True, fmt='.2%', cmap='YlOrRd', ax=ax3)
ax3.set_title('P(Estado de Actividad | Préstamo Bancario)')
ax3.tick_params(axis='x', rotation=45)
ax3.tick_params(axis='y', rotation=45)

# 4. P(Región | Estado de actividad)
sns.heatmap(prob_bayes['p_region_dado_act'], annot=True, fmt='.2%', cmap='YlOrRd', ax=ax4)
ax4.set_title('P(Región | Estado de Actividad)')
ax4.tick_params(axis='x', rotation=45)
ax4.tick_params(axis='y', rotation=45)

plt.tight_layout()
plt.show()

# Mostrar algunos resultados interesantes
print("P(Nivel educativo superior completo | Ocupado):")
print(f"{prob_bayes['p_educ_dado_act'].loc['Superior universitario completo', 'Ocupado']:.1%}")

print("\nP(Ocupado | Tiene préstamo bancario):")
print(f"{prob_bayes['p_act_dado_prestamo_banco'].loc['Ocupado', 'Sí']:.1%}")

print("\nP(GBA | Desocupado):")
print(f"{prob_bayes['p_region_dado_act'].loc['GBA', 'Desocupado']:.1%}")


In [ ]:
# Modelado con distribuciones teóricas
def ajustar_distribuciones(datos):
    resultados = {}
    
    # Distribución Log-normal para ingresos
    ingresos_positivos = datos[datos['ingresos'] > 0]['ingresos']
    log_ingresos = np.log(ingresos_positivos)
    mu_log, sigma_log = stats.norm.fit(log_ingresos)
    resultados['lognorm_ingresos'] = {
        'mu': mu_log,
        'sigma': sigma_log,
        'x': np.linspace(ingresos_positivos.min(), ingresos_positivos.max(), 100),
        'empirica': ingresos_positivos
    }
    
    # Distribución Normal para edad
    mu_edad, sigma_edad = stats.norm.fit(datos['edad'])
    resultados['norm_edad'] = {
        'mu': mu_edad,
        'sigma': sigma_edad,
        'x': np.linspace(datos['edad'].min(), datos['edad'].max(), 100),
        'empirica': datos['edad']
    }
    
    # Distribución Poisson para ocupados por hogar
    lambda_ocup = datos['ocupados_hogar'].mean()
    resultados['poisson_ocupados'] = {
        'lambda': lambda_ocup,
        'x': np.arange(0, datos['ocupados_hogar'].max() + 1),
        'empirica': datos['ocupados_hogar']
    }
    
    # Distribución Bernoulli para acceso a crédito
    for credito in ['compra_cuotas', 'prestamo_personas', 'prestamo_banco']:
        p = (datos[credito] == 'Sí').mean()
        resultados[f'bernoulli_{credito}'] = {
            'p': p,
            'empirica': datos[credito] == 'Sí'
        }
    
    return resultados

# Ajustar distribuciones
dist = ajustar_distribuciones(datos)

# Visualizaciones
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Distribución Log-normal de ingresos
sns.histplot(data=np.log(dist['lognorm_ingresos']['empirica']), stat='density', 
             bins=50, alpha=0.5, ax=ax1)
ax1.plot(np.log(dist['lognorm_ingresos']['x']), 
         stats.norm.pdf(np.log(dist['lognorm_ingresos']['x']), 
                       dist['lognorm_ingresos']['mu'], 
                       dist['lognorm_ingresos']['sigma']),
         'r-', lw=2, label='Log-normal teórica')
ax1.set_title('Distribución de Log-Ingresos')
ax1.set_xlabel('Log(Ingresos)')
ax1.set_ylabel('Densidad')
ax1.legend()

# 2. Distribución Normal de edad
sns.histplot(data=dist['norm_edad']['empirica'], stat='density', 
             bins=50, alpha=0.5, ax=ax2)
ax2.plot(dist['norm_edad']['x'], 
         stats.norm.pdf(dist['norm_edad']['x'], 
                       dist['norm_edad']['mu'], 
                       dist['norm_edad']['sigma']),
         'r-', lw=2, label='Normal teórica')
ax2.set_title('Distribución de Edad')
ax2.set_xlabel('Edad')
ax2.set_ylabel('Densidad')
ax2.legend()

# 3. Distribución Poisson de ocupados por hogar
sns.histplot(data=dist['poisson_ocupados']['empirica'], stat='probability',
             bins=len(dist['poisson_ocupados']['x']), alpha=0.5, ax=ax3)
ax3.plot(dist['poisson_ocupados']['x'], 
         stats.poisson.pmf(dist['poisson_ocupados']['x'], 
                          dist['poisson_ocupados']['lambda']),
         'ro-', lw=2, label='Poisson teórica')
ax3.set_title('Distribución de Ocupados por Hogar')
ax3.set_xlabel('Número de Ocupados')
ax3.set_ylabel('Probabilidad')
ax3.legend()

# 4. Distribución Bernoulli de acceso a crédito
creditos = ['compra_cuotas', 'prestamo_personas', 'prestamo_banco']
props = [dist[f'bernoulli_{c}']['p'] for c in creditos]
emp_props = [(dist[f'bernoulli_{c}']['empirica']).mean() for c in creditos]

x = np.arange(len(creditos))
width = 0.35
ax4.bar(x - width/2, props, width, label='Bernoulli teórica')
ax4.bar(x + width/2, emp_props, width, label='Proporción empírica')
ax4.set_title('Distribución de Acceso a Crédito')
ax4.set_xticks(x)
ax4.set_xticklabels(['Cuotas', 'Préstamo\nPersonal', 'Préstamo\nBancario'])
ax4.set_ylabel('Probabilidad')
ax4.legend()

plt.tight_layout()
plt.show()

# Mostrar parámetros estimados
print("Parámetros estimados:")
print(f"\nIngresos (Log-normal):")
print(f"μ = {dist['lognorm_ingresos']['mu']:.2f}")
print(f"σ = {dist['lognorm_ingresos']['sigma']:.2f}")

print(f"\nEdad (Normal):")
print(f"μ = {dist['norm_edad']['mu']:.2f}")
print(f"σ = {dist['norm_edad']['sigma']:.2f}")

print(f"\nOcupados por hogar (Poisson):")
print(f"λ = {dist['poisson_ocupados']['lambda']:.2f}")

print("\nAcceso a crédito (Bernoulli):")
for credito in creditos:
    print(f"{credito}: p = {dist[f'bernoulli_{credito}']['p']:.2f}")


In [ ]:
# Pruebas de bondad de ajuste
def realizar_pruebas_ajuste(datos, dist):
    resultados = {}
    
    # Test KS para ingresos (log-normal)
    log_ingresos = np.log(dist['lognorm_ingresos']['empirica'])
    ks_ingresos = stats.kstest(log_ingresos, 'norm',
                              args=(dist['lognorm_ingresos']['mu'],
                                   dist['lognorm_ingresos']['sigma']))
    resultados['ks_ingresos'] = ks_ingresos
    
    # Test KS para edad (normal)
    ks_edad = stats.kstest(dist['norm_edad']['empirica'], 'norm',
                          args=(dist['norm_edad']['mu'],
                                dist['norm_edad']['sigma']))
    resultados['ks_edad'] = ks_edad
    
    # Test Chi-cuadrado para ocupados (Poisson)
    obs_freq = np.bincount(dist['poisson_ocupados']['empirica'].astype(int))
    exp_freq = stats.poisson.pmf(np.arange(len(obs_freq)), 
                                dist['poisson_ocupados']['lambda']) * len(dist['poisson_ocupados']['empirica'])
    chi2_ocupados = stats.chisquare(obs_freq, exp_freq)
    resultados['chi2_ocupados'] = chi2_ocupados
    
    # Test Chi-cuadrado para acceso a crédito (Bernoulli)
    for credito in ['compra_cuotas', 'prestamo_personas', 'prestamo_banco']:
        obs = np.bincount(dist[f'bernoulli_{credito}']['empirica'].astype(int))
        p = dist[f'bernoulli_{credito}']['p']
        exp = np.array([1-p, p]) * len(dist[f'bernoulli_{credito}']['empirica'])
        chi2_credito = stats.chisquare(obs, exp)
        resultados[f'chi2_{credito}'] = chi2_credito
    
    return resultados

# Realizar pruebas
pruebas = realizar_pruebas_ajuste(datos, dist)

# Visualizaciones
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. QQ-plot para log-ingresos
stats.probplot(np.log(dist['lognorm_ingresos']['empirica']), 
               dist='norm', plot=ax1)
ax1.set_title('QQ-Plot Log-Ingresos')

# 2. QQ-plot para edad
stats.probplot(dist['norm_edad']['empirica'], 
               dist='norm', plot=ax2)
ax2.set_title('QQ-Plot Edad')

# 3. Residuos para ocupados (Poisson)
obs_freq = np.bincount(dist['poisson_ocupados']['empirica'].astype(int))
exp_freq = stats.poisson.pmf(np.arange(len(obs_freq)), 
                            dist['poisson_ocupados']['lambda']) * len(dist['poisson_ocupados']['empirica'])
residuos = (obs_freq - exp_freq) / np.sqrt(exp_freq)
ax3.bar(np.arange(len(residuos)), residuos)
ax3.axhline(y=0, color='r', linestyle='-')
ax3.set_title('Residuos Estandarizados - Ocupados por Hogar')
ax3.set_xlabel('Número de Ocupados')
ax3.set_ylabel('Residuo Estandarizado')

# 4. Residuos para acceso a crédito (Bernoulli)
creditos = ['compra_cuotas', 'prestamo_personas', 'prestamo_banco']
residuos_credito = []
for credito in creditos:
    obs = np.bincount(dist[f'bernoulli_{credito}']['empirica'].astype(int))
    p = dist[f'bernoulli_{credito}']['p']
    exp = np.array([1-p, p]) * len(dist[f'bernoulli_{credito}']['empirica'])
    residuo = (obs - exp) / np.sqrt(exp)
    residuos_credito.append(residuo[1])  # Solo mostramos residuo para "Sí"

ax4.bar(np.arange(len(creditos)), residuos_credito)
ax4.axhline(y=0, color='r', linestyle='-')
ax4.set_title('Residuos Estandarizados - Acceso a Crédito')
ax4.set_xticks(np.arange(len(creditos)))
ax4.set_xticklabels(['Cuotas', 'Préstamo\nPersonal', 'Préstamo\nBancario'])
ax4.set_ylabel('Residuo Estandarizado')

plt.tight_layout()
plt.show()

# Mostrar resultados de las pruebas
print("Resultados de las pruebas de bondad de ajuste:")

print("\nTest Kolmogorov-Smirnov para Log-Ingresos:")
print(f"Estadístico = {pruebas['ks_ingresos'].statistic:.4f}")
print(f"p-valor = {pruebas['ks_ingresos'].pvalue:.4e}")

print("\nTest Kolmogorov-Smirnov para Edad:")
print(f"Estadístico = {pruebas['ks_edad'].statistic:.4f}")
print(f"p-valor = {pruebas['ks_edad'].pvalue:.4e}")

print("\nTest Chi-cuadrado para Ocupados por Hogar:")
print(f"Estadístico = {pruebas['chi2_ocupados'].statistic:.4f}")
print(f"p-valor = {pruebas['chi2_ocupados'].pvalue:.4e}")

print("\nTest Chi-cuadrado para Acceso a Crédito:")
for credito in creditos:
    print(f"\n{credito}:")
    print(f"Estadístico = {pruebas[f'chi2_{credito}'].statistic:.4f}")
    print(f"p-valor = {pruebas[f'chi2_{credito}'].pvalue:.4e}")


In [1]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configuración de visualización
plt.style.use('seaborn')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

# Configurar visualización de datos en el notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Intentar cargar datos procesados o procesarlos desde los archivos originales
try:
    print("Intentando cargar datos procesados...")
    datos = pd.read_pickle('datos_procesados.pkl')
except FileNotFoundError:
    print("Datos procesados no encontrados. Procesando datos desde archivos originales...")
    from preparar_datos_eph import cargar_datos
    datos = cargar_datos()

# Mostrar información básica del dataset
print("\nInformación del dataset:")
print(datos.info())

print("\nPrimeras filas:")
datos.head()


In [ ]:
# Probabilidades simples
def calcular_probabilidades_simples(datos):
    resultados = {}
    
    # P(Nivel educativo)
    resultados['p_nivel_educ'] = datos['nivel_educativo'].value_counts(normalize=True)
    
    # P(Desocupado)
    resultados['p_desoc'] = datos['desocupado'].mean()
    
    # P(Compra en cuotas)
    resultados['p_compra_cuotas'] = (datos['compra_en_cuotas'] == 'Sí').mean()
    
    # P(Universitario Y Desocupado)
    resultados['p_univ_desoc'] = ((datos['nivel_educativo'] == 'Superior universitario completo') & 
                                 (datos['desocupado'] == 1)).mean()
    
    # P(Universitario O Desocupado)
    resultados['p_univ_o_desoc'] = ((datos['nivel_educativo'] == 'Superior universitario completo') | 
                                   (datos['desocupado'] == 1)).mean()
    
    # P(Región)
    resultados['p_region'] = datos['region'].value_counts(normalize=True)
    
    return resultados

# Calcular probabilidades
prob_simples = calcular_probabilidades_simples(datos)

# Mostrar resultados
print("Probabilidades por nivel educativo:")
print(prob_simples['p_nivel_educ'])
print(f"\nProbabilidad de estar desocupado: {prob_simples['p_desoc']:.3f}")
print(f"Probabilidad de comprar en cuotas: {prob_simples['p_compra_cuotas']:.3f}")
print(f"P(Universitario ∩ Desocupado): {prob_simples['p_univ_desoc']:.3f}")
print(f"P(Universitario ∪ Desocupado): {prob_simples['p_univ_o_desoc']:.3f}")

# Visualizar distribuciones
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Nivel educativo
prob_simples['p_nivel_educ'].plot(kind='bar', ax=ax1)
ax1.set_title('Distribución de Nivel Educativo')
ax1.set_xlabel('Nivel Educativo')
ax1.set_ylabel('Probabilidad')
ax1.tick_params(axis='x', rotation=45)

# Región
prob_simples['p_region'].plot(kind='bar', ax=ax2)
ax2.set_title('Distribución por Región')
ax2.set_xlabel('Región')
ax2.set_ylabel('Probabilidad')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Calcular probabilidades condicionales por nivel educativo
def calcular_prob_condicionales(datos):
    # P(Desocupado | Nivel educativo)
    p_desoc_dado_nivel = datos.groupby('nivel_educativo')['desocupado'].mean()
    
    # P(Compra en cuotas | Nivel educativo)
    p_compra_dado_nivel = datos.groupby('nivel_educativo').apply(
        lambda x: (x['compra_en_cuotas'] == 'Sí').mean()
    )
    
    # P(Desocupado | Región)
    p_desoc_dado_region = datos.groupby('region')['desocupado'].mean()
    
    return p_desoc_dado_nivel, p_compra_dado_nivel, p_desoc_dado_region

# Calcular probabilidades
p_desoc_nivel, p_compra_nivel, p_desoc_region = calcular_prob_condicionales(datos)

# Visualizar resultados
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Gráfico de desocupación por nivel educativo
p_desoc_nivel.plot(kind='bar', ax=ax1)
ax1.set_title('P(Desocupado | Nivel Educativo)')
ax1.set_xlabel('Nivel Educativo')
ax1.set_ylabel('Probabilidad')
ax1.tick_params(axis='x', rotation=45)

# Gráfico de compra en cuotas por nivel educativo
p_compra_nivel.plot(kind='bar', ax=ax2)
ax2.set_title('P(Compra en cuotas | Nivel Educativo)')
ax2.set_xlabel('Nivel Educativo')
ax2.set_ylabel('Probabilidad')
ax2.tick_params(axis='x', rotation=45)

# Gráfico de desocupación por región
p_desoc_region.plot(kind='bar', ax=ax3)
ax3.set_title('P(Desocupado | Región)')
ax3.set_xlabel('Región')
ax3.set_ylabel('Probabilidad')
ax3.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Mostrar valores numéricos
print("P(Desocupado | Nivel Educativo):")
print(p_desoc_nivel)
print("\nP(Compra en cuotas | Nivel Educativo):")
print(p_compra_nivel)
print("\nP(Desocupado | Región):")
print(p_desoc_region)


In [ ]:
# Probabilidades simples
def calcular_probabilidades_simples(datos):
    resultados = {}
    
    # P(Nivel educativo)
    resultados['p_nivel_educ'] = datos['nivel_educativo'].value_counts(normalize=True)
    
    # P(Desocupado)
    resultados['p_desoc'] = datos['desocupado'].mean()
    
    # P(Sin baño)
    resultados['p_sin_banio'] = (datos['baño'] == 0).mean()
    
    # P(Universitario Y Desocupado)
    resultados['p_univ_desoc'] = ((datos['nivel_educativo'] == 'Universitario completo') & 
                                 (datos['desocupado'] == 1)).mean()
    
    # P(Universitario O Desocupado)
    resultados['p_univ_o_desoc'] = ((datos['nivel_educativo'] == 'Universitario completo') | 
                                   (datos['desocupado'] == 1)).mean()
    
    return resultados

# Calcular probabilidades
prob_simples = calcular_probabilidades_simples(datos)

# Mostrar resultados
print("Probabilidades por nivel educativo:")
print(prob_simples['p_nivel_educ'])
print(f"\nProbabilidad de estar desocupado: {prob_simples['p_desoc']:.3f}")
print(f"Probabilidad de no tener baño: {prob_simples['p_sin_banio']:.3f}")
print(f"P(Universitario ∩ Desocupado): {prob_simples['p_univ_desoc']:.3f}")
print(f"P(Universitario ∪ Desocupado): {prob_simples['p_univ_o_desoc']:.3f}")

# Visualizar distribución de nivel educativo
plt.figure(figsize=(12, 6))
prob_simples['p_nivel_educ'].plot(kind='bar')
plt.title('Distribución de Nivel Educativo')
plt.xlabel('Nivel Educativo')
plt.ylabel('Probabilidad')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
def verificar_independencia(datos, var1, var2, val1, val2, nombres=None):
    """
    Verifica independencia entre dos variables
    """
    if nombres is None:
        nombres = {'var1': var1, 'var2': var2, 'val1': val1, 'val2': val2}
    
    # Probabilidades marginales
    p_a = (datos[var1] == val1).mean()
    p_b = (datos[var2] == val2).mean()
    
    # Probabilidad conjunta
    p_ab = ((datos[var1] == val1) & (datos[var2] == val2)).mean()
    
    # Producto de probabilidades marginales
    p_a_x_p_b = p_a * p_b
    
    # Diferencia absoluta
    diferencia = abs(p_ab - p_a_x_p_b)
    
    print(f"Test de independencia entre {nombres['var1']} y {nombres['var2']}:")
    print(f"P({nombres['val1']}) = {p_a:.4f}")
    print(f"P({nombres['val2']}) = {p_b:.4f}")
    print(f"P({nombres['val1']} ∩ {nombres['val2']}) = {p_ab:.4f}")
    print(f"P({nombres['val1']}) * P({nombres['val2']}) = {p_a_x_p_b:.4f}")
    print(f"Diferencia absoluta = {diferencia:.4f}")
    print(f"Conclusión: Los eventos {'son' if diferencia < 0.01 else 'no son'} independientes\n")
    
    return diferencia < 0.01

# Verificar independencia entre diferentes pares de variables
# 1. Educación universitaria y desocupación
verificar_independencia(
    datos,
    'nivel_educativo',
    'desocupado',
    'Superior universitario completo',
    1,
    {'var1': 'Educación universitaria', 'var2': 'Desocupación',
     'val1': 'Univ. completo', 'val2': 'Desocupado'}
)

# 2. Educación universitaria y compra en cuotas
verificar_independencia(
    datos,
    'nivel_educativo',
    'compra_en_cuotas',
    'Superior universitario completo',
    'Sí',
    {'var1': 'Educación universitaria', 'var2': 'Compra en cuotas',
     'val1': 'Univ. completo', 'val2': 'Compra en cuotas'}
)

# 3. Región y desocupación
verificar_independencia(
    datos,
    'region',
    'desocupado',
    'Gran Buenos Aires',
    1,
    {'var1': 'Región', 'var2': 'Desocupación',
     'val1': 'GBA', 'val2': 'Desocupado'}
)


In [ ]:
def aplicar_bayes(datos, condicion, evento, nombres=None):
    """
    Aplica el teorema de Bayes para calcular P(evento|condicion)
    """
    if nombres is None:
        nombres = {'evento': evento[1], 'condicion': condicion[1]}
    
    # P(evento)
    p_evento = (datos[evento[0]] == evento[1]).mean()
    
    # P(condicion)
    p_condicion = (datos[condicion[0]] == condicion[1]).mean()
    
    # P(condicion|evento)
    p_condicion_dado_evento = datos[datos[evento[0]] == evento[1]][condicion[0]].mean()
    
    # Teorema de Bayes: P(evento|condicion) = P(condicion|evento) * P(evento) / P(condicion)
    p_evento_dado_condicion = (p_condicion_dado_evento * p_evento) / p_condicion if p_condicion > 0 else 0
    
    print(f"Teorema de Bayes - P({nombres['evento']}|{nombres['condicion']}):")
    print(f"P({nombres['evento']}) = {p_evento:.4f}")
    print(f"P({nombres['condicion']}) = {p_condicion:.4f}")
    print(f"P({nombres['condicion']}|{nombres['evento']}) = {p_condicion_dado_evento:.4f}")
    print(f"P({nombres['evento']}|{nombres['condicion']}) = {p_evento_dado_condicion:.4f}\n")
    
    return p_evento_dado_condicion

# 1. P(Universitario|Desocupado)
bayes_univ_desoc = aplicar_bayes(
    datos,
    ('desocupado', 1),
    ('nivel_educativo', 'Superior universitario completo'),
    {'evento': 'Univ. completo', 'condicion': 'Desocupado'}
)

# 2. P(Universitario|Compra en cuotas)
bayes_univ_cuotas = aplicar_bayes(
    datos,
    ('compra_en_cuotas', 'Sí'),
    ('nivel_educativo', 'Superior universitario completo'),
    {'evento': 'Univ. completo', 'condicion': 'Compra en cuotas'}
)

# 3. P(GBA|Desocupado)
bayes_gba_desoc = aplicar_bayes(
    datos,
    ('desocupado', 1),
    ('region', 'Gran Buenos Aires'),
    {'evento': 'GBA', 'condicion': 'Desocupado'}
)


In [ ]:
# Calcular probabilidades condicionales por nivel educativo
def calcular_prob_condicionales(datos):
    # P(Desocupado | Nivel educativo)
    p_desoc_dado_nivel = datos.groupby('nivel_educativo')['desocupado'].mean()
    
    # P(Sin baño | Nivel educativo)
    p_sin_banio_dado_nivel = datos.groupby('nivel_educativo').apply(
        lambda x: (x['baño'] == 0).mean()
    )
    
    return p_desoc_dado_nivel, p_sin_banio_dado_nivel

# Calcular probabilidades
p_desoc_nivel, p_sin_banio_nivel = calcular_prob_condicionales(datos)

# Visualizar resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de desocupación por nivel educativo
p_desoc_nivel.plot(kind='bar', ax=ax1)
ax1.set_title('P(Desocupado | Nivel Educativo)')
ax1.set_xlabel('Nivel Educativo')
ax1.set_ylabel('Probabilidad')
ax1.tick_params(axis='x', rotation=45)

# Gráfico de falta de baño por nivel educativo
p_sin_banio_nivel.plot(kind='bar', ax=ax2)
ax2.set_title('P(Sin Baño | Nivel Educativo)')
ax2.set_xlabel('Nivel Educativo')
ax2.set_ylabel('Probabilidad')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Mostrar valores numéricos
print("P(Desocupado | Nivel Educativo):")
print(p_desoc_nivel)
print("\nP(Sin Baño | Nivel Educativo):")
print(p_sin_banio_nivel)


In [ ]:
# Modelado de ocupados por hogar (Poisson)
lambda_ocupados = datos['ocupados_hogar'].mean()
poisson_ocupados = stats.poisson(lambda_ocupados)

# Modelado de ingresos (Normal)
mu_ingresos = datos['ingresos'].mean()
sigma_ingresos = datos['ingresos'].std()
normal_ingresos = stats.norm(mu_ingresos, sigma_ingresos)

# Modelado de edad (Normal)
mu_edad = datos['edad'].mean()
sigma_edad = datos['edad'].std()
normal_edad = stats.norm(mu_edad, sigma_edad)

# Calcular probabilidades
print("Modelado con distribuciones:")
print("-" * 50)

# Poisson - Ocupados por hogar
p_mas_2_ocupados = 1 - poisson_ocupados.cdf(2)
print(f"P(Ocupados > 2) según Poisson = {p_mas_2_ocupados:.3f}")
print(f"λ (media de ocupados por hogar) = {lambda_ocupados:.2f}")

# Normal - Ingresos
umbral_alto = mu_ingresos + sigma_ingresos
p_ingreso_alto = 1 - normal_ingresos.cdf(umbral_alto)
print(f"\nP(Ingreso > μ + σ) según Normal = {p_ingreso_alto:.3f}")
print(f"μ (media de ingresos) = {mu_ingresos:.2f}")
print(f"σ (desviación estándar de ingresos) = {sigma_ingresos:.2f}")

# Normal - Edad
p_edad_joven = normal_edad.cdf(30)
print(f"\nP(Edad ≤ 30) según Normal = {p_edad_joven:.3f}")
print(f"μ (media de edad) = {mu_edad:.2f}")
print(f"σ (desviación estándar de edad) = {sigma_edad:.2f}")

# Bernoulli - Variables binarias
print("\nModelos Bernoulli:")
p_desocupado = datos['desocupado'].mean()
p_compra_cuotas = (datos['compra_en_cuotas'] == 'Sí').mean()
print(f"P(Desocupado) = {p_desocupado:.3f}")
print(f"P(Compra en cuotas) = {p_compra_cuotas:.3f}")


In [ ]:
# Crear subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Ocupados por hogar (empírico vs Poisson)
sns.histplot(data=datos, x='ocupados_hogar', stat='probability', 
             discrete=True, label='Empírico', ax=ax1)

x = np.arange(0, datos['ocupados_hogar'].max() + 1)
poisson_pmf = stats.poisson.pmf(x, lambda_ocupados)
ax1.plot(x, poisson_pmf, 'r-', label='Poisson teórica')

ax1.set_title('Distribución de ocupados por hogar')
ax1.set_xlabel('Número de ocupados')
ax1.set_ylabel('Probabilidad')
ax1.legend()

# 2. Ingresos (empírico vs Normal)
sns.histplot(data=datos, x='ingresos', stat='density', 
             label='Empírico', ax=ax2)

x = np.linspace(datos['ingresos'].min(), datos['ingresos'].max(), 100)
normal_pdf = stats.norm.pdf(x, mu_ingresos, sigma_ingresos)
ax2.plot(x, normal_pdf, 'r-', label='Normal teórica')

ax2.set_title('Distribución de ingresos')
ax2.set_xlabel('Ingreso')
ax2.set_ylabel('Densidad')
ax2.legend()

# 3. Edad (empírico vs Normal)
sns.histplot(data=datos, x='edad', stat='density', 
             label='Empírico', ax=ax3)

x = np.linspace(datos['edad'].min(), datos['edad'].max(), 100)
normal_pdf = stats.norm.pdf(x, mu_edad, sigma_edad)
ax3.plot(x, normal_pdf, 'r-', label='Normal teórica')

ax3.set_title('Distribución de edad')
ax3.set_xlabel('Edad')
ax3.set_ylabel('Densidad')
ax3.legend()

# 4. Variables Bernoulli
variables_bernoulli = {
    'Desocupado': p_desocupado,
    'Compra en cuotas': p_compra_cuotas
}
ax4.bar(variables_bernoulli.keys(), variables_bernoulli.values())
ax4.set_title('Probabilidades de variables Bernoulli')
ax4.set_ylabel('Probabilidad')
ax4.set_ylim(0, 1)

plt.tight_layout()
plt.show()

# Pruebas de bondad de ajuste
print("Pruebas de bondad de ajuste:")
print("-" * 50)

# Test Kolmogorov-Smirnov para ingresos (Normal)
ks_stat_ing, ks_pval_ing = stats.kstest(
    (datos['ingresos'] - mu_ingresos) / sigma_ingresos,
    'norm'
)
print(f"Test KS para normalidad de ingresos:")
print(f"Estadístico: {ks_stat_ing:.4f}")
print(f"Valor p: {ks_pval_ing:.4f}")

# Test Kolmogorov-Smirnov para edad (Normal)
ks_stat_edad, ks_pval_edad = stats.kstest(
    (datos['edad'] - mu_edad) / sigma_edad,
    'norm'
)
print(f"\nTest KS para normalidad de edad:")
print(f"Estadístico: {ks_stat_edad:.4f}")
print(f"Valor p: {ks_pval_edad:.4f}")

# Test Chi-cuadrado para ocupados por hogar (Poisson)
observed = datos['ocupados_hogar'].value_counts().sort_index()
expected = stats.poisson.pmf(observed.index, lambda_ocupados) * len(datos)
chi2_stat, chi2_pval = stats.chisquare(observed, expected)
print(f"\nTest Chi-cuadrado para Poisson de ocupados:")
print(f"Estadístico: {chi2_stat:.4f}")
print(f"Valor p: {chi2_pval:.4f}")


In [ ]:
def verificar_independencia(datos, var1, var2, val1, val2):
    """
    Verifica independencia entre dos variables binarias
    """
    # Probabilidades marginales
    p_a = (datos[var1] == val1).mean()
    p_b = (datos[var2] == val2).mean()
    
    # Probabilidad conjunta
    p_ab = ((datos[var1] == val1) & (datos[var2] == val2)).mean()
    
    # Producto de probabilidades marginales
    p_a_x_p_b = p_a * p_b
    
    # Diferencia absoluta
    diferencia = abs(p_ab - p_a_x_p_b)
    
    return {
        'P(A)': p_a,
        'P(B)': p_b,
        'P(A∩B)': p_ab,
        'P(A)*P(B)': p_a_x_p_b,
        'Diferencia': diferencia,
        'Son independientes': diferencia < 0.01
    }

# Verificar independencia entre nivel universitario y desocupación
resultados = verificar_independencia(
    datos,
    'nivel_educativo',
    'desocupado',
    'Universitario completo',
    1
)

print("Test de independencia entre educación universitaria y desocupación:")
for k, v in resultados.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")

# Verificar independencia entre nivel universitario y acceso a baño
resultados_banio = verificar_independencia(
    datos,
    'nivel_educativo',
    'baño',
    'Universitario completo',
    0
)

print("\nTest de independencia entre educación universitaria y falta de baño:")
for k, v in resultados_banio.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")


In [ ]:
def aplicar_bayes(datos, condicion, evento):
    """
    Aplica el teorema de Bayes para calcular P(evento|condicion)
    """
    # P(evento)
    p_evento = (datos[evento[0]] == evento[1]).mean()
    
    # P(condicion)
    p_condicion = (datos[condicion[0]] == condicion[1]).mean()
    
    # P(condicion|evento)
    p_condicion_dado_evento = datos[datos[evento[0]] == evento[1]][condicion[0]].mean()
    
    # Teorema de Bayes: P(evento|condicion) = P(condicion|evento) * P(evento) / P(condicion)
    p_evento_dado_condicion = (p_condicion_dado_evento * p_evento) / p_condicion
    
    return {
        f'P({evento[1]})': p_evento,
        f'P({condicion[1]})': p_condicion,
        f'P({condicion[1]}|{evento[1]})': p_condicion_dado_evento,
        f'P({evento[1]}|{condicion[1]})': p_evento_dado_condicion
    }

# Calcular P(Universitario|Desocupado)
bayes_desoc = aplicar_bayes(
    datos,
    ('desocupado', 1),
    ('nivel_educativo', 'Universitario completo')
)

print("Teorema de Bayes - P(Universitario|Desocupado):")
for k, v in bayes_desoc.items():
    print(f"{k}: {v:.4f}")

# Calcular P(Universitario|Sin baño)
bayes_banio = aplicar_bayes(
    datos,
    ('baño', 0),
    ('nivel_educativo', 'Universitario completo')
)

print("\nTeorema de Bayes - P(Universitario|Sin baño):")
for k, v in bayes_banio.items():
    print(f"{k}: {v:.4f}")


In [ ]:
# Modelado de ocupados por hogar (Poisson)
lambda_ocupados = datos['ocupados_hogar'].mean()
poisson_ocupados = stats.poisson(lambda_ocupados)

# Modelado de ingresos (Normal)
mu_ingresos = datos['ingresos'].mean()
sigma_ingresos = datos['ingresos'].std()
normal_ingresos = stats.norm(mu_ingresos, sigma_ingresos)

# Calcular probabilidades
print("Modelado con distribuciones:")
print("-" * 50)

# Poisson - Ocupados por hogar
p_mas_2_ocupados = 1 - poisson_ocupados.cdf(2)
print(f"P(Ocupados > 2) según Poisson = {p_mas_2_ocupados:.3f}")
print(f"λ (media de ocupados por hogar) = {lambda_ocupados:.2f}")

# Normal - Ingresos
umbral_alto = mu_ingresos + sigma_ingresos
p_ingreso_alto = 1 - normal_ingresos.cdf(umbral_alto)
print(f"\nP(Ingreso > μ + σ) según Normal = {p_ingreso_alto:.3f}")
print(f"μ (media de ingresos) = {mu_ingresos:.2f}")
print(f"σ (desviación estándar de ingresos) = {sigma_ingresos:.2f}")

# Bernoulli - Desocupación y baño
p_desocupado = datos['desocupado'].mean()
p_sin_banio = (datos['baño'] == 0).mean()
print(f"\nP(Desocupado) según Bernoulli = {p_desocupado:.3f}")
print(f"P(Sin baño) según Bernoulli = {p_sin_banio:.3f}")


In [ ]:
# Crear subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Subplot 1: Ocupados por hogar (empírico vs Poisson)
sns.histplot(data=datos, x='ocupados_hogar', stat='probability', 
             discrete=True, label='Empírico', ax=ax1)

x = np.arange(0, datos['ocupados_hogar'].max() + 1)
poisson_pmf = stats.poisson.pmf(x, lambda_ocupados)
ax1.plot(x, poisson_pmf, 'r-', label='Poisson teórica')

ax1.set_title('Distribución de ocupados por hogar')
ax1.set_xlabel('Número de ocupados')
ax1.set_ylabel('Probabilidad')
ax1.legend()

# Subplot 2: Ingresos (empírico vs Normal)
sns.histplot(data=datos, x='ingresos', stat='density', 
             label='Empírico', ax=ax2)

x = np.linspace(datos['ingresos'].min(), datos['ingresos'].max(), 100)
normal_pdf = stats.norm.pdf(x, mu_ingresos, sigma_ingresos)
ax2.plot(x, normal_pdf, 'r-', label='Normal teórica')

ax2.set_title('Distribución de ingresos')
ax2.set_xlabel('Ingreso')
ax2.set_ylabel('Densidad')
ax2.legend()

plt.tight_layout()
plt.show()

# Pruebas de bondad de ajuste
print("Pruebas de bondad de ajuste:")
print("-" * 50)

# Test Kolmogorov-Smirnov para ingresos (Normal)
ks_stat, ks_pval = stats.kstest(
    (datos['ingresos'] - mu_ingresos) / sigma_ingresos,
    'norm'
)
print(f"Test KS para normalidad de ingresos:")
print(f"Estadístico: {ks_stat:.4f}")
print(f"Valor p: {ks_pval:.4f}")

# Test Chi-cuadrado para ocupados por hogar (Poisson)
observed = datos['ocupados_hogar'].value_counts().sort_index()
expected = stats.poisson.pmf(observed.index, lambda_ocupados) * len(datos)
chi2_stat, chi2_pval = stats.chisquare(observed, expected)
print(f"\nTest Chi-cuadrado para Poisson de ocupados:")
print(f"Estadístico: {chi2_stat:.4f}")
print(f"Valor p: {chi2_pval:.4f}")
